<h1> Einleitung </h1>

<div class="alert alert-info">
  <strong>In meinem Projekt möchte ich die Beziehung zwischen der Breitband-Internetgeschwindigkeit beziehungsweise Internetnutzung und dem BIP pro Land untersuchen. Die Daten stammen heirbei von der Website Kaggle.
    </strong>
</div>

## Aufbau

<div class="alert alert-success">
  <strong>   
  <ol>
      <li>Importieren der Packages/Datasets</li>
      <li>Korrelationsanalysis</li>
      <li>Resultat</li>
  </ol>
  </strong>
</div>

### 1. Importieren der Packages/Datasets

<blockquote>
    Zuerst müssen die Packages und Extensions geladen werden, die ich für meine Berechnung brauchen werde:
</blockquote>

In [1]:
import requests
import sqlite3 
#%pip install pandas #uncomment if needed
import pandas as pd
#%pip install SPARQLWrapper #uncomment if needed
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
#%pip install scipy #uncomment if needed
from scipy.stats import pearsonr

<blockquote>
    Hier lade ich die Daten aus meinem Github-Repo mit dem Modul 'request' rein, da ich bei Kaggle keine API gefunden habe.
</blockquote>

In [2]:
r = requests.get('https://raw.githubusercontent.com/NinaMirey/datasets-for-correlation-analysis/main/broadband_usage.csv')
f = open('broadband_usage.csv', 'wb')
f.write(r.content)
f.close()

<blockquote>
    Dann lade ich SQl und verbinde ich mich zu meiner Database names 'internet_gdp.db'. 
</blockquote>

In [3]:
%reload_ext sql
%sql sqlite:///internet_gdp.db

In [4]:
conn = sqlite3.connect('internet_gdp.db')

<blockquote>
    Hier lade ich die CSV-Files in 'internet_gpd' rein und zeige zur Kontrolle 5 Reihen davon an.
</blockquote>

In [5]:
pd.read_csv('broadband_usage.csv').to_sql('internet_gdp', conn, if_exists='replace',index = False)

167

In [6]:
%sql select * from internet_gdp limit 5;

 * sqlite:///internet_gdp.db
Done.


country,avgBroadbandMbps,internetUsers,population,userPercent
Monaco,192.68,37553,38695,97.05%
Singapore,192.01,4821119,5708844,84.45%
Chile,189.36,14864456,18054726,82.33%
Thailand,184.03,36513941,69037513,52.89%
Denmark,163.6,5567278,5733551,97.10%


In [7]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

<blockquote>
    Folgende sparql-query sucht jene Länder in Englisch, die einen BIP haben. Mit der for-Schleife gehe ich durch und printe das Ergebnis dann:
</blockquote>

In [8]:
sparql.setQuery("""select ?countryl ?gdp 
WHERE {
?country a dbo:Country .
?country rdfs:label ?countryl .
?country dbp:gdpNominalPerCapita ?gdp .
filter(datatype(?gdp) = 'http://dbpedia.org/datatype/usDollar')
FILTER (lang(?countryl) = 'en')}
ORDER BY ?countryl""") 

sparql.setReturnFormat(JSON)
data = sparql.query().convert()

for res in data["results"]["bindings"]:
    results = (res["countryl"]["value"], res["gdp"]["value"])
    print(results)

('ABC islands (Leeward Antilles)', '47113.0')
('ASEAN', '5376.0')
('Afghanistan', '493.0')
('African Union', '1958.0')
('Albania', '6260.0')
('Algeria', '4151.0')
('Andorra', '41930.0')
('Angola', '3793.0')
('Antigua and Barbuda', '18416.0')
('Arab League', '6600.0')
('Arab Maghreb Union', '3720.0')
('Argentina', '13622.0')
('Armenia', '4595.0')
('Aruba', '28898.0')
('Australia', '66408.0')
('Austria', '53320.0')
('Azerbaijan', '5398.0')
("Ba'athist Iraq", '761.0')
('Bahrain', '29103.0')
('Bangladesh', '2734.0')
('Barbados', '18133.0')
('Belarus', '6133.0')
('Belgium', '50598.0')
('Belize', '4890.0')
('Benin', '971.0')
('Bermuda', '117097.0')
('Bhutan', '3500.0')
('Bolivia', '3823.0')
('Bosnia and Herzegovina', '7078.0')
('Botswana', '7519.0')
('Brazil', '8570.0')
('British Hong Kong', '24698.0')
('Brunei', '42939.0')
('Bulgaria', '12340.0')
('Burkina Faso', '926.0')
('Burundi', '310.0')
('Cambodia', '1752.0')
('Cameroon', '1649.0')
('Canada', '56794.0')
('Canton 10', '5.762')
('Cape V

<blockquote>
    Nun habe ich BIPs, die geordnet nach Ländern sind. Jedoch will ich diese Daten schöner darstellen. Dafür importiere ich das Package 'display' und erstelle eine formatierte HTML-Tabelle.
</blockquote>

In [9]:
from IPython.display import HTML, display

tab = '<table>'
tab += '<h1 style="color:powderblue;">Land mit BIP</h1><table>\n'
for res in data["results"]["bindings"]:
    tab += '<tr style="background-color: rgba(150, 212, 212, 0.4);"><td style="width:150px";>%s<td>%s' % (res["countryl"]["value"], res["gdp"]["value"])
display(HTML(tab+'</table>'))

<blockquote>
    Sieht gut aus - Nun parse ich diese Daten in JSON-Format und zeige sie zur Kontrolle an, um nachher damit meine Berechnungen durchzuführen:
</blockquote>

In [10]:
def json2sql(jsn, tablename, conn):
    head = jsn['head']['vars']
    rows = [[ row[head[i]]['value'] for i in range(len(head))] for row in jsn['results']['bindings'] ]
    conn.cursor().execute("drop table if exists " + tablename)
    pd.DataFrame(rows, columns=head).to_sql(tablename, conn, index = False)

json2sql(data, 'data', conn)

In [11]:
%sql select * from data limit 5;

 * sqlite:///internet_gdp.db
Done.


countryl,gdp
ABC islands (Leeward Antilles),47113.0
ASEAN,5376.0
Afghanistan,493.0
African Union,1958.0
Albania,6260.0


## 2. Korrelationsanalyse

<blockquote>
    Nun tue ich jene Spalten zusammenfassen, die für meine Berechnung interessant sind und tu die nach dem BIP ordnen. <i>Man kann möglicherweise schon mit bloßem Auge erkennen, welche Werte eine Zusammenhang vorweisen:</i>
</blockquote>

#### Übersicht von interessanten Daten:

<blockquote>
<b>Country:</b> Um welches Land handelt es sich?<br>
<b>AvgBroadbandMbps:</b> Durchschnittliche Bandbreitegeschwindigkeit<br>
<b>UserPercent:</b> Wie viel Internetnutzer hat jedes Land in Prozent?<br>
<b>Gpd:</b> Welches BIP hat das Land?<br>
</blockquote>

In [12]:
%%sql
SELECT country, gdp, userPercent, avgBroadbandMbps from internet_gdp join data on data.countryl = internet_gdp.country
ORDER BY CAST(data.gdp AS INTEGER) DESC;

 * sqlite:///internet_gdp.db
Done.


country,gdp,userPercent,avgBroadbandMbps
Monaco,190513.0,97.05%,192.68
Liechtenstein,180000.0,98.10%,118.19
Luxembourg,135046.0,97.83%,92.1
Norway,92646.0,96.51%,105.61
Switzerland,92434.0,93.71%,110.3
Qatar,82877.0,95.94%,72.22
Singapore,79576.0,84.45%,192.01
United States,75180.0,75.23%,143.76
Denmark,68094.0,97.10%,163.6
Australia,66408.0,86.54%,50.89


<blockquote>
    Man erkennt, finde ich, dass es Verbindungen zwischen dem BIP und der Internutzung bzw. Brandbreitegeschwindigkeit gibt - sind diese jedoch signifikant?<br> <u>Das sollten wir nachrechen:</u>
</blockquote>

In [13]:
#zur Sichherheit nochmal eine Verbindung herstellen
conn = sqlite3.connect('internet_gdp.db')

#Abfrage für die Berechnung definieren(ohne %) und gefilterte Daten in unser dataframe laden
query = """
SELECT data.gdp, 
    REPLACE(internet_gdp.userPercent, '%', '') AS userPercent, 
    internet_gdp.avgBroadbandMbps
FROM internet_gdp
JOIN data ON data.countryl = internet_gdp.country
"""
df = pd.read_sql_query(query, conn)

# in Fließkommazahl umwandeln, damit man damit rechenn kann
df['gdp'] = pd.to_numeric(df['gdp'], errors='coerce')
df['avgBroadbandMbps'] = pd.to_numeric(df['avgBroadbandMbps'], errors='coerce')
df['userPercent'] = pd.to_numeric(df['userPercent'], errors='coerce')

#Korrelation zwischen Bip und Internetnutzeranteil mit dem pearsonr-module 
correlation_percentage = pearsonr(df['gdp'], df['userPercent'])
print(f"Korrelation zw. BIP und Inernet-User-Anteil: {correlation_percentage}")
#Korrelation zwischen Bip und Brandbreitegeschwindigkeit
correlation_broadband = pearsonr(df['gdp'], df['avgBroadbandMbps'])
print(f"Korrelation zw. BIP und Brandbreitegeschwindigkeit: {correlation_broadband}")

#meine Berechnungen sind fertig, also schließe ich die Verbindung
conn.close()

Korrelation zw. BIP und Inernet-User-Anteil: PearsonRResult(statistic=0.6372887290469644, pvalue=7.69014881340538e-20)
Korrelation zw. BIP und Brandbreitegeschwindigkeit: PearsonRResult(statistic=0.644201390043412, pvalue=2.2867527787324018e-20)


## 3. Resultat

<div class="alert alert-success"> 
  <ul>
      <li>Die Korrelation zwischen dem <i>BIP</i> und dem <i>Anteil der Internetnutzer</i> beträgt aufgerundet <i>0,64</i>, was auch auf eine moderate positive Korrelation hinweist. Der p-Wert ist hierbei mit ca. <i>7.69e-20</i> sehr klein.</li><br>
      <li>Die Korrelation zwischen dem <i>BIP</i> und der <i>Breitbandgeschwindigkeit</i> beträgt abgerundet ebenfalls <i>0,64</i>. Der p-Wert ist ebenfalls mit <i>2.29e-20</i> klein.</li><br>
      <li>Bei beide Berechnungen ist der p-Wert <i>unter 0,05</i>, was bedeutet, dass diese <i>sehr signifikant</i> sind</li><br>
      <li><b>In beiden Fällen ist es also sehr wahrscheinlich keine zufällige Beziehung, sondern es gibt eine tatsächliche Verbindung.</b></li><br>
  </ul>
</div>

#### <u>Meine Begründung:</u>

<blockquote>
<div class="alert alert-info">
  <strong>Wirtschaftlich stärkere Länder verfügen über mehr finanzielle Mittel, was zu einer höheren Verfügbarkeit von Internetdiensten führt. Breitband-Internet ermöglicht schnellere Datenübertragung, was für viele moderne Arbeitsweisen essentiell ist.
  </strong>
</div>
</blockquote>

 <b> 
    The end c:
</b>